In [11]:
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.special

In [12]:
N_CHANNEL_STATES = 10
TAU_LIMIT = 9
DISCOUNT_FACTOR = 0.6
WEIGHING_FACTOR = 1

In [13]:
class Environment:
    def __init__(self) -> None:
        self.time = 1
        self.A = 1.2
        self.Q = 1
        self.C = 1
        self.R = 1
        self.sigma_2 = 8 # Noise power
        self.packet_length = 10 
        self.P_bar = 0.6613
        self.zeta = 45 # Transmitted energy

        self.state = random.randint(0, N_CHANNEL_STATES*(TAU_LIMIT + 1))

        self.channel_gains = np.array(range(10)) * 0.2 + 0.2
        self.eta = np.array([[.91,.09,  0,  0,  0,  0,  0,  0,  0,  0],
                             [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0],
                             [ .8,.12,.08,  0,  0,  0,  0,  0,  0,  0],
                             [ .7,.11,.11,.08,  0,  0,  0,  0,  0,  0],
                             [ .6,.11, .1, .1,.09,  0,  0,  0,  0,  0],
                             [ .5,.18, .1,.08,.07,.07,  0,  0,  0,  0],
                             [ .4,.28, .1,.06,.05,.06,.05,  0,  0,  0],
                             [ .3, .1, .1, .1,.16,.14,.09,.01,  0,  0],
                             [ .2,.11, .1, .1, .1,.05,.09,.13,.12,  0],
                             [ .1, .1, .1, .1, .1, .1, .1, .1, .1, .1]])
        self.h_tau_P = np.zeros(10)
        self.h_tau_P[0] = self.P_bar
        for i in range(10):
            if i == 0:
                continue
            self.h_tau_P[i] = self.A ** 2 * self.h_tau_P[i-1]
        print(self.h_tau_P)
        print(self.channel_gains)

        self.p_js = np.zeros(N_CHANNEL_STATES)
        for j in range(N_CHANNEL_STATES):
            SNR = self.channel_gains[j] * self.zeta / self.sigma_2
            self.p_js[j] = self.f(SNR)

        
    def robot_step(self, action):
        tau = self.state // N_CHANNEL_STATES
        h = self.state % N_CHANNEL_STATES

        self.time += 1
        self.next_state(action)

        return self.h_tau_P[tau] + action * WEIGHING_FACTOR * self.zeta

    def f(self, SNR):   # Calculates the probability of successful transmission based on SNR
        sqrt_SNR = np.sqrt(SNR * 2)
        I = scipy.special.erf(sqrt_SNR)
        
        return I**self.packet_length

    def next_state(self, action):
        tau = self.state // N_CHANNEL_STATES
        h = self.state % N_CHANNEL_STATES

        new_h = random.choices(population=range(N_CHANNEL_STATES), weights=self.eta[h, :], k=1)[0]
        new_tau = tau + 1
        if action == 1:
            r = random.uniform(0,1)
            SNR = self.channel_gains[h] * self.zeta / self.sigma_2
            prob_packet_transmission = self.f(SNR)
            if r < prob_packet_transmission:
                new_tau = 0
        
        if new_tau > TAU_LIMIT: # Ceiling functionality
            new_tau = TAU_LIMIT
        
        self.state = (new_tau * N_CHANNEL_STATES) + new_h
        
    def reset(self):
        self.time = 1
        self.state = random.randint(0, N_CHANNEL_STATES - 1)

    def reset_to_explore_starts(self):
        self.time = 1
        self.state = random.randint(0, N_CHANNEL_STATES * (TAU_LIMIT + 1) - 1)

    def evaluate_policy(self, policy):
        runs = 100
        length_of_episode = 200
        discount = 1
        reward = 0
        for run in range(runs):
            self.reset()
            discount = 1
            for t in range(length_of_episode):
                reward += self.robot_step(policy[self.state]) * discount
                discount *= DISCOUNT_FACTOR
                # print(reward, discount)
        return reward / (runs)

In [14]:
env = Environment()

[ 0.66  0.95  1.37  1.97  2.84  4.09  5.9   8.49 12.23 17.61]
[0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2. ]


In [15]:
class MDP:
    def __init__(self) -> None:
        self.policy = np.array(np.random.rand(N_CHANNEL_STATES * (TAU_LIMIT + 1)) > 0.5, dtype=np.int32)
            # This is initially a random policy, which will be finalized after values are calculated
        self.estimated_costs = np.ones(shape=(N_CHANNEL_STATES * (TAU_LIMIT + 1), 2)) * 1000 
            # This is to understand the estimated cost of taking an action in a given state, 
            # which is initialized to a very high value to prevent usage of unexplored states
        self.count_of_transitions = np.zeros(shape=(2, N_CHANNEL_STATES * (TAU_LIMIT + 1), N_CHANNEL_STATES * (TAU_LIMIT + 1)))
            # This is the count of transitions from state s to state s' given action a
        self.estimate_transitions = np.zeros(shape=(2, N_CHANNEL_STATES * (TAU_LIMIT + 1), N_CHANNEL_STATES * (TAU_LIMIT + 1)))
            # This is the normalized probability of the transition s -> s' given action a
        self.estimate_values = np.zeros(N_CHANNEL_STATES * (TAU_LIMIT + 1))
            # Once we calculate the transitions, this is our current estimate of the value of each state
        self.n_states = N_CHANNEL_STATES * (TAU_LIMIT + 1)
    def calculate_transitions(self, env):
        env.reset_to_explore_starts()
        for run in range(1000):
            # Exploring starts with any state starting
            env.reset_to_explore_starts()
            # To generate episodes
            for t in range(1000):
                # Interact with the environment
                s = env.state
                a = random.randint(0,1)
                r = env.robot_step(action=a)
                s_dash = env.state

                # If this is the first exploration of the state, update the estimated cost :
                if self.count_of_transitions[a, s, s_dash] == 0:
                    self.estimated_costs[s, a] = r

                # Update counts:
                self.count_of_transitions[a, s, s_dash] += 1
        
        # Normalize the counts to get transition probabilities:
        for s in range(N_CHANNEL_STATES * (TAU_LIMIT + 1)):
            for a in range(2):
                if np.sum(self.count_of_transitions[a, s, :]) != 0:
                    self.estimate_transitions[a, s, :] = self.count_of_transitions[a, s, :] / np.sum(self.count_of_transitions[a, s, :])
        print(self.estimate_transitions.shape)

    def calculate_transitions_2(self, env):
        self.new_transitions = np.zeros(shape=(2, N_CHANNEL_STATES * (TAU_LIMIT + 1), N_CHANNEL_STATES * (TAU_LIMIT + 1)))
        for s in range(self.n_states - N_CHANNEL_STATES):
            tau = s // N_CHANNEL_STATES
            h = s % N_CHANNEL_STATES

            new_tau = (tau + 1)
            for new_h in range(10):
                new_state_0 = (new_tau * N_CHANNEL_STATES) + new_h
                new_state_1 = (tau * N_CHANNEL_STATES) + new_h
                if new_state_0 < self.n_states:
                    self.new_transitions[0, s, new_state_0] = env.eta[h, new_h]
                    self.new_transitions[1, s, new_state_0] = env.eta[h, new_h] * (1 - env.p_js[h])
                self.new_transitions[1, s, new_state_1] = env.eta[h, new_h] * env.p_js[h]
            c = env.h_tau_P[tau]
            self.estimated_costs[s, 0] = c
            self.estimated_costs[s, 1] = c + env.zeta
        
        
        for s in range(self.n_states - N_CHANNEL_STATES, self.n_states):
            tau = s // N_CHANNEL_STATES
            h = s % N_CHANNEL_STATES
            for new_h in range(10):
                new_state = (tau * N_CHANNEL_STATES) + new_h
                self.new_transitions[1, s, new_state] = env.eta[h, new_h] * env.p_js[h]
            c = env.h_tau_P[tau]
            self.estimated_costs[s, 1] = c + env.zeta


    def value_iter_2(self):
        count = 0
        for i in range(100):
            count += 1
            opt1 = self.estimated_costs[:, 0] + DISCOUNT_FACTOR * np.reshape(self.new_transitions[0, :, :], newshape=(self.n_states, self.n_states)) @ self.estimate_values
                # Estimated value of choosing to not transmit (action = 0)
            opt2 = self.estimated_costs[:, 1] + DISCOUNT_FACTOR * np.reshape(self.new_transitions[1, :, :], newshape=(self.n_states, self.n_states)) @ self.estimate_values
                # Estimated value of choosing to transmit (action = 1)
            
            self.policy = np.argmin(np.array([opt1, opt2]), axis=0)

            new_vals = np.max(np.array([opt1, opt2]), axis=0)

            if np.linalg.norm(new_vals - self.estimate_values) < 1:
                break
            self.estimate_values = new_vals

    def value_iter(self):
        count = 0
        for i in range(100):
            count += 1
            opt1 = self.estimated_costs[:, 0] + DISCOUNT_FACTOR * np.reshape(self.estimate_transitions[0, :, :], newshape=(self.n_states, self.n_states)) @ self.estimate_values
                # Estimated value of choosing to not transmit (action = 0)
            opt2 = self.estimated_costs[:, 1] + DISCOUNT_FACTOR * np.reshape(self.estimate_transitions[1, :, :], newshape=(self.n_states, self.n_states)) @ self.estimate_values
                # Estimated value of choosing to transmit (action = 1)
            
            self.policy = np.argmin(np.array([opt1, opt2]), axis=0)

            new_vals = np.max(np.array([opt1, opt2]), axis=0)

            if np.linalg.norm(new_vals - self.estimate_values) < 1:
                break
            self.estimate_values = new_vals
        # print(count)
    def print_policy(self):
        print(np.reshape(self.policy, newshape=(N_CHANNEL_STATES, TAU_LIMIT + 1)))


In [16]:
p_MDP = MDP()

np.set_printoptions(precision=2)


In [17]:
p_MDP.calculate_transitions_2(env)

In [18]:
p_MDP.value_iter_2()
# print(p_MDP.new_transitions[0, 10:20, 20:30])
p_MDP.print_policy()

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


In [19]:
env.evaluate_policy(p_MDP.policy)

5.8501629994905215